Sveučilište u Zagrebu  
Fakultet elektrotehnike i računarstva  
  
## Strojno učenje 1 2021/2022
http://www.fer.unizg.hr/predmet/struce1

------------------------------

### Probabilistički grafički modeli

*Verzija: 1.0  
Zadnji put ažurirano: 6. siječnja 2021.*

(c) 2015-2021 Jan Šnajder, Domagoj Alagić  

Rok za predaju: **9. siječnja 2021. u 23:59h**

------------------------------

### Upute

Ova laboratorijska vježba sastoji se od dva zadatka. U nastavku slijedite upute navedene u ćelijama s tekstom. Rješavanje vježbe svodi se na **dopunjavanje ove bilježnice**: umetanja ćelije ili više njih **ispod** teksta zadatka, pisanja odgovarajućeg kôda te evaluiranja ćelija. 

Osigurajte da u potpunosti **razumijete** kôd koji ste napisali. Kod predaje vježbe, morate biti u stanju na zahtjev asistenta (ili demonstratora) preinačiti i ponovno evaluirati Vaš kôd. Nadalje, morate razumjeti teorijske osnove onoga što radite, u okvirima onoga što smo obradili na predavanju. Ispod nekih zadataka možete naći i pitanja koja služe kao smjernice za bolje razumijevanje gradiva (**nemojte pisati** odgovore na pitanja u bilježnicu). Stoga se nemojte ograničiti samo na to da riješite zadatak, nego slobodno eksperimentirajte. To upravo i jest svrha ovih vježbi.

Vježbe trebate raditi **samostalno**. Možete se konzultirati s drugima o načelnom načinu rješavanja, ali u konačnici morate sami odraditi vježbu. U protivnome vježba nema smisla.

In [1]:
# Učitaj osnovne biblioteke...
import sklearn
from sklearn.metrics import silhouette_samples, silhouette_score
import numpy as np
import matplotlib.pyplot as plt
import pgmpy as pgm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### 1. Probabilistički grafički modeli -- Bayesove mreže

Ovaj zadatak bavit će se Bayesovim mrežama, jednim od poznatijih probabilističkih grafičkih modela (*probabilistic graphical models*; PGM). Za lakše eksperimentiranje koristit ćemo programski paket [`pgmpy`](https://github.com/pgmpy/pgmpy). Molimo Vas da provjerite imate li ovaj paket te da ga instalirate ako ga nemate. Upute se nalaze na gornjoj poveznici. Za korisnike Anaconde, najlakše je upisati `conda install -c ankurankan pgmpy` (ili `pip install pgmpy` ako ne prolazi) unutar Anaconda Prompta (i ponovno pokrenuti Jupyter).

Prvo ćemo pogledati udžbenički primjer s prskalicom. U ovom primjeru razmatramo Bayesovu mrežu koja modelira zavisnosti između oblačnosti (slučajna varijabla $C$), kiše ($R$), prskalice ($S$) i mokre trave ($W$). U ovom primjeru također pretpostavljamo da već imamo parametre vjerojatnosnih distribucija svih čvorova. Ova mreža prikazana je na sljedećoj slici:

![This](http://www.fer.unizg.hr/_download/repository/bayes-net-sprinkler.jpg)

Koristeći paket `pgmpy`, konstruirajte Bayesovu mrežu iz gornjeg primjera. Zatim, koristeći **egzaktno** zaključivanje, postavite sljedeće posteriorne upite: $P(w=1)$, $P(s=1|w=1)$, $P(r=1|w=1)$, $P(c=1|s=1, r=1)$ i $P(c=1)$. Provedite zaključivanje na papiru i uvjerite se da ste ispravno konstruirali mrežu. Pomoći će vam službena dokumentacija te primjeri korištenja (npr. [ovaj](https://github.com/pgmpy/pgmpy/blob/dev/examples/Monty%20Hall%20Problem.ipynb)).

In [2]:
from pgmpy.models import BayesianModel
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination

In [11]:
#egzaktno zaključivanje = (u ovom slučaju) eliminacija varijabli

## konstrukcija Bayesove mreže
#cloudy = C, S = sprinkler, R = rain, W = wet grass
model = BayesianNetwork([("C", "S"), ("C", "R"), ("S","W"), ("R","W")])
# cloudy nema nikakvih zavisnosti
cpd_c = TabularCPD("C", 2, [[0.50], [0.50]])
# evidence card je broj mogućnih stanja u "dokazima"
# npr. ako je evidence varijabla cloudy, evidence card = 2, jer je cloudy binarna varijabl
cpd_r = TabularCPD("R",2,[[0.8, 0.20],[0.20, 0.80]], evidence=["C"], evidence_card=[2])
cpd_s = TabularCPD("S",2,[[0.10, 0.50],[0.90, 0.50]], evidence=["C"], evidence_card=[2])
cpd_w = TabularCPD("W",2, [[0.99, 0.90, 0.90, 0], [0.01, 0.10, 0.10, 1]], evidence=["S","R"],evidence_card=[2,2])

# Associating the CPDs with the network structure.
model.add_cpds(cpd_c, cpd_r, cpd_s, cpd_w)
# check_model check for the model structure and the associated CPD and returns True if everything is correct otherwise throws an exception
model.check_model()

## zakljucivanje
infer = VariableElimination(model)
print("Egzaktno zakljulčivanje za upit P(W=1)")
posterior_prob = infer.query(["W"])
print(posterior_prob)
print("##################################")

print("Egzaktno zaključivanje za upit P(S=1|W=1)")
posterior_prob = infer.query(["S"], evidence={"W":1})
print(posterior_prob)
print("##################################")

print("Egzaktno zaključivanje za upit P(R=1|W=1)")
posterior_prob = infer.query(["R"], evidence={"W":1})
print(posterior_prob)
print("##################################")

print("Egzaktno zaključivanje za upit P(C=1|S=1, R=1)")
posterior_prob = infer.query(["C"], evidence={"S":1, "R":1})
print(posterior_prob)
print("##################################")

print("Egzaktno zaključivanje za upit P(C=1)")
posterior_prob = infer.query(["C"])
print(posterior_prob)
print("##################################")

Egzaktno zakljulčivanje za upit P(W=1)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

+------+----------+
| W    |   phi(W) |
+======+==========+
| W(0) |   0.6471 |
+------+----------+
| W(1) |   0.3529 |
+------+----------+
##################################
Egzaktno zaključivanje za upit P(S=1|W=1)


  0%|          | 0/2 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.3103 |
+------+----------+
| C(1) |   0.6897 |
+------+----------+
##################################
Egzaktno zaključivanje za upit P(C=1)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

+------+----------+
| C    |   phi(C) |
+======+==========+
| C(0) |   0.5000 |
+------+----------+
| C(1) |   0.5000 |
+------+----------+
##################################


**Q:** Koju zajedničku vjerojatnosnu razdiobu ova mreža modelira? Kako tu informaciju očitati iz mreže?  
**Q:** U zadatku koristimo egzaktno zaključivanje. Kako ono radi?  
**Q:** Koja je razlika između posteriornog upita i MAP-upita?  
**Q:** Zašto je vjerojatnost $P(c=1)$ drugačija od $P(c=1|s=1,r=1)$ ako znamo da čvorovi $S$ i $R$ nisu roditelji čvora $C$?

### 2. Efekt objašnjavanja

 **Efekt objašnjavanja** (engl. *explaining away*) zanimljiv je fenomen u kojem se događa da se dvije varijable "natječu" za objašnjavanje treće. Ovaj fenomen može se primijetiti na gornjoj mreži. U tom se slučaju varijable prskalice ($S$) i kiše ($R$) "natječu" za objašnjavanje mokre trave ($W$). Vaš zadatak je pokazati da se fenomen zaista događa.

In [14]:
# cilj - pokazati da se varijable prskalice, S i kiše, R natječu za objašnjavanje mokre trave
# S i R svaka samostalno mogu objasniti W
# ako znamo da se ostvario W, naše vjerovanje o tome je li se ostvario npr. S se mijenja 
# ovisno o tome je li se ostvario R, i obratno

# promatramo S
# 1) znamo da se dogodio W, ne znamo ništa o R
print("1 - dogodio se W, ne znamo ništa o R")
posterior_prob = infer.query(["S"], evidence={"W":1})
print(posterior_prob)

# 2) znamo da se dogodio W, znamo da se dogodio i R
print("2 - dogodio se W, znamo da se dogodio i R")
posterior_prob = infer.query(["S"], evidence={"W":1, "R":1})
print(posterior_prob)

# 3) znamo da se dogodio W, znamo da se NIJE dogodio i R
print("3 - dogodio se W, znamo da se NIJE dogodio i R")
posterior_prob = infer.query(["S"], evidence={"W":1, "R":0})
print(posterior_prob)

# vidimo da kad znamo da se nije dogodio R, povećava se vjerojatnost da 
# se dogodio S, jer ako nije, kako je inače trava mokra (bez kiše i prskalice)
# tj. vidimo da se naše vjerovanje je li se ostvario S, ako znamo da se ostvario W, mijenja ovisno o tome
# znamo li je li se ostvario R

# promatramo R
# 4) znamo da se dogodio W, ne znamo ništa o S
print("4 - dogodio se W, ne znamo ništa o S")
posterior_prob = infer.query(["R"], evidence={"W":1})
print(posterior_prob)

# 5) znamo da se dogodio W, znamo da se dogodio S
print("5 - dogodio se W, znamo da se dogodio i S")
posterior_prob = infer.query(["R"], evidence={"W":1, "S":1})
print(posterior_prob)

# 6) znamo da se dogodio W, znamo da se NIJE dogodio S
print("6 - dogodio se W, znamo da se NIJE dogodio S")
posterior_prob = infer.query(["R"], evidence={"W":1, "S":0})
print(posterior_prob)

# opet, primijećujemo da se vjerojatnost da se dogodio R povećava ako znamo 
# da se nije dogodio S, a da se dogodio W u odnosu na slučaj kad znamo da se dogodio W, a da se dogodio S
# opet, jer moramo nekako objasniti mokru travu




1 - dogodio se W, ne znamo ništa o R


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.0621 |
+------+----------+
| S(1) |   0.9379 |
+------+----------+
2 - dogodio se W, znamo da se dogodio i R


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.1239 |
+------+----------+
| R(1) |   0.8761 |
+------+----------+
6 - dogodio se W, znamo da se NIJE dogodio S


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.0411 |
+------+----------+
| R(1) |   0.9589 |
+------+----------+


**Q:** Kako biste svojim riječima opisali ovaj fenomen, koristeći se ovim primjerom?